# Understanding HTTP 

  * The Hypertext Transfer Protocol (HTTP) is how your browser communicates with the internet. 
  * HTTP is a client/server protocol. 
  * The *client* is the browser. 
  * The *server* is software like Flask. 
  * The client initiates all requests 
  * The server responds to requests 
    * The server never contacts the client directly. 
  * In a typical HTTP session the client requests a *resource* from the server. 

Suppose you have the following flask program:

In [ ]:
from flask import Flask

app = Flask(__name__)

@app.route('/')
def hello() :
    return "The server says hello."

if __name__ == '__main__' :
    app.run(port=8080)

When you direct your browser to your application the browser sends this request: 

<code>
GET / HTTP/1.1
Host: localhost:5050
Connection: keep-alive
User-Agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36
</code>

Flask replies with a server response like this: 

<code>
HTTP/1.0 200 OK
Content-Type: text/html; charset=utf-8
Content-Length: 22
Server: Werkzeug/0.14.1 Python/3.6.2
Date: Sun, 25 Feb 2018 22:32:57 GMT

The server says hello.
</code> 

The most essential line in the request is:

<code>
GET / HTTP/1.1
</code>

  * That's the client telling the server what it wants. 
  * What comes after the `GET` is the page part of the URL. 
  * For example when your browser retrieved this page it asked for:

<code>
GET /cis-15/user_input HTTP/1.1
</code>

  * The server always sends back a response code. 
    * You're probably familiar with the most famous one.
  * The response code tells the client what happened to the request 

Here's a table of the most used response codes: 

| Code | Meaning | 
| ---- | ------- | 
| 200 | OK - The response contains the page requested | 
| 301 | Moved Permanently - The server wants to redirect you to a different page. | 
| 400 | Bad Request | 
| 403 | Forbidden | 
| 404 | Page not found. | 
| 500 | Server error | 

## Flask Debugging Output 

  * When your flask program is running, flask supplies information about every request. 
  * The request and response information is written to the command line. 

Here's an example of Flask's output: 

<code>
127.0.0.1 - - [25/Feb/2018 14:32:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2018 14:32:57] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [25/Feb/2018 15:01:38] "GET /bogus HTTP/1.1" 404 -
</code>

  * The first line shows a successful request (200) 
  * The second line shows the browser attempting to find an icon for the site (404) 
  * The third request is for a bogus page (404)
 
# HTML Forms 

  * An HTML form is an HTML document that takes input from the user. 
  * When the user presses "Submit" the form data is sent to the web server
  * Flask can handle form data
  * If you're new to HTML forms read this tutorial with examples
    * https://www.w3schools.com/html/html_forms.asp

Here's an example of a simple form that could have been used for Project 3: 

<code html>
<html>
    <head>
        <title>Simple Form</title>
    </head>
    <body>
        <form action="/result" method="post">
          Value for the variable a:<br>
          <input type="text" name="a_value"><br>
          Value for the variable b:<br>
          <input type="text" name="b_value"><br><br>
          <input type="submit" value="Submit">
        </form>
    </body>
</html>
</code>

  * Notice the following about the form:
    * It's `action` field is set to `/result`
      * The forms action will be directed to `<the-form-url>/result` 
      * The application must have a `route` to this URL 
    * The `method` is set to `post`.
      * Submitting this form will create an HTTP `POST` request.
    * The text inputs are named `a_value` and `b_value`. Those will appear in the Python program
    * The last input is the Submit button.

The form is handled by the following Python function:

In [ ]:
@app.route('/result', methods=['POST'])
def do_table():
    a = float(request.form['a_value'])
    b = float(request.form['b_value'])
    data = []
    data.append(['a', a])
    data.append(['b', b])
    data.append([f'{a}+{b}', a+b])
    return render_template('basic_answer.html', table_data=data)

  * Notice the following. 
    * The form field values are accessible by the variable `request.form` which is a dictionary 
    * The `@app.route` decorator has an additional argument `methods=['POST']` 
      * That allows the function to handle post data. 
      * The form would be rejected otherwise. 

You need a function to render the form. That's shown below:

In [ ]:
@app.route('/', methods=['GET'])
def do_form():
    return render_template('basic_form.html')

The code simply renders the form template, which is not parameterizable. 

<callout type="danger" icon="True"> 
# HTML Form Security Warning 
The forms in this section aren't secure. Secure forms should: 
  - Validate user input 
  - Contain a random number to prevent https://en.wikipedia.org/wiki/Cross-site_request_forgery|cross site request forgery (XSRF)
I will show you how to do forms securely in the coming weeks! 
</callout>

# Routing GET Requests 

  * POST forms are "hide" the information they contain inside the HTTP request. 
  * GET request can use the URL to encode data. 
  * For example to find the Reddit user `foobar` you can go to their URL
    * https://www.reddit.com/user/foobar/
  * Flask can bind parts of a URL to function arguments 

Here's a function that gets it's arguments from the URL

In [ ]:
@app.route('/math/<a>/<b>')
def do_table(a, b):
    data = []
    data.append(['a', a])
    data.append(['b', b])
    data.append([f'{a}+{b}', a+b])
    return render_template('basic_answer.html', table_data=data)

  * Taking input this way is recommended in situations where
    * You want browser bookmarks to work 
    * You want the forward and back buttons on the browser to do the right thing